In [2]:
# load packages
import networkx as nx
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline


## Data Selection

In [3]:
# extract data
route_url = 'https://raw.githubusercontent.com/eddiexunyc/web_analytics_work/refs/heads/main/Project%201/Resources/routes.csv'
airplane_url = 'https://raw.githubusercontent.com/eddiexunyc/web_analytics_work/refs/heads/main/Project%201/Resources/airplanes.csv'

route_data = pd.read_csv(route_url)
airplane_data = pd.read_csv(airplane_url)

## Exploratory Data Analysis